# Bike Sharing Dataset
## Data Cleaning & Vector Space Representation

In [13]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler


## Load Dataset

In [14]:

df = pd.read_csv("bike_sharing_dataset/hour.csv")
df.head()


,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


## Initial Inspection

In [15]:

df.info()
df.describe()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17379 entries, 0 to 17378
Data columns (total 17 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   instant     17379 non-null  int64  
 1   dteday      17379 non-null  object 
 2   season      17379 non-null  int64  
 3   yr          17379 non-null  int64  
 4   mnth        17379 non-null  int64  
 5   hr          17379 non-null  int64  
 6   holiday     17379 non-null  int64  
 7   weekday     17379 non-null  int64  
 8   workingday  17379 non-null  int64  
 9   weathersit  17379 non-null  int64  
 10  temp        17379 non-null  float64
 11  atemp       17379 non-null  float64
 12  hum         17379 non-null  float64
 13  windspeed   17379 non-null  float64
 14  casual      17379 non-null  int64  
 15  registered  17379 non-null  int64  
 16  cnt         17379 non-null  int64  
dtypes: float64(4), int64(12), object(1)
memory usage: 2.3+ MB


,instant,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
count,17379.0000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000
mean,8690.0000,2.501640,0.502561,6.537775,11.546752,0.028770,3.003683,0.682721,1.425283,0.496987,0.475775,0.627229,0.190098,35.676218,153.786869,189.463088
std,5017.0295,1.106918,0.500008,3.438776,6.914405,0.167165,2.005771,0.465431,0.639357,0.192556,0.171850,0.192930,0.122340,49.305030,151.357286,181.387599
min,1.0000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,4345.5000,2.000000,0.000000,4.000000,6.000000,0.000000,1.000000,0.000000,1.000000,0.340000,0.333300,0.480000,0.104500,4.000000,34.000000,40.000000
50%,8690.0000,3.000000,1.000000,7.000000,12.000000,0.000000,3.000000,1.000000,1.000000,0.500000,0.484800,0.630000,0.194000,17.000000,115.000000,142.000000
75%,13034.5000,3.000000,1.000000,10.000000,18.000000,0.000000,5.000000,1.000000,2.000000,0.660000,0.621200,0.780000,0.253700,48.000000,220.000000,281.000000
max,17379.0000,4.000000,1.000000,12.000000,23.000000,1.000000,6.000000,1.000000,4.000000,1.000000,1.000000,1.000000,0.850700,367.000000,886.000000,977.000000


In [16]:
# Missing values
print(df.isna().sum())
print("\nMissing values (%):")
print((df.isna().sum() / len(df) * 100).round(2))

# Duplicates
print(f"\nTotal duplicate rows: {df.duplicated().sum()}")
print(f"Duplicates (keep first): {df.duplicated(keep='first').sum()}")

# Remove duplicates if needed
df = df.drop_duplicates(keep='first')

instant       0
dteday        0
season        0
yr            0
mnth          0
hr            0
holiday       0
weekday       0
workingday    0
weathersit    0
temp          0
atemp         0
hum           0
windspeed     0
casual        0
registered    0
cnt           0
dtype: int64

Missing values (%):
instant       0.0
dteday        0.0
season        0.0
yr            0.0
mnth          0.0
hr            0.0
holiday       0.0
weekday       0.0
workingday    0.0
weathersit    0.0
temp          0.0
atemp         0.0
hum           0.0
windspeed     0.0
casual        0.0
registered    0.0
cnt           0.0
dtype: float64

Total duplicate rows: 0
Duplicates (keep first): 0


## Data Cleaning

In [17]:
# Drop non-informative columns
# instant: just a sequence index, no predictive value
# dteday: date string, hour already captures temporal patterns

df_clean = df.drop(columns=["instant", "dteday"])

print(f"Columns after cleaning: {df_clean.columns.tolist()}")
print(f"Shape: {df_clean.shape}")

Columns after cleaning: ['season', 'yr', 'mnth', 'hr', 'holiday', 'weekday', 'workingday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed', 'casual', 'registered', 'cnt']
Shape: (17379, 15)


In [18]:
# Validate data integrity for data consistency issues: casual users + registered users should equal total count
validation_error = (df_clean["casual"] + df_clean["registered"] - df_clean["cnt"]).abs().sum()

print(f"Data integrity check (should be 0): {validation_error}")

if validation_error == 0:
    print("Data is consistent: casual + registered = cnt")
else:
    print("Data is inconsistent!")

Data integrity check (should be 0): 0
Data is consistent: casual + registered = cnt


## Encode Categorical Variables

In [19]:
# Encode categorical variables to numerical format
df_encoded = pd.get_dummies(
    df_clean,
    columns=["season", "weathersit", "weekday"]
)

print(f"Columns after encoding: {df_encoded.columns.tolist()}")
print(f"Shape after encoding: {df_encoded.shape}")
print(f"\nData types:\n{df_encoded.dtypes}")

Columns after encoding: ['yr', 'mnth', 'hr', 'holiday', 'workingday', 'temp', 'atemp', 'hum', 'windspeed', 'casual', 'registered', 'cnt', 'season_1', 'season_2', 'season_3', 'season_4', 'weathersit_1', 'weathersit_2', 'weathersit_3', 'weathersit_4', 'weekday_0', 'weekday_1', 'weekday_2', 'weekday_3', 'weekday_4', 'weekday_5', 'weekday_6']
Shape after encoding: (17379, 27)

Data types:
yr                int64
mnth              int64
hr                int64
holiday           int64
workingday        int64
temp            float64
atemp           float64
hum             float64
windspeed       float64
casual            int64
registered        int64
cnt               int64
season_1           bool
season_2           bool
season_3           bool
season_4           bool
weathersit_1       bool
weathersit_2       bool
weathersit_3       bool
weathersit_4       bool
weekday_0          bool
weekday_1          bool
weekday_2          bool
weekday_3          bool
weekday_4          bool
weekday_5   

## Feature Scaling

In [20]:
# Standardize numerical features to zero mean and unit variance - this could be an optional step
# Note: 'hr' is kept unscaled as it's cyclical (others can be as well if decided that it is unnecessary)
scaler = StandardScaler()

numerical_cols = [
    "temp", "atemp", "hum", "windspeed",
    "cnt", "casual", "registered"
]

df_encoded[numerical_cols] = scaler.fit_transform(
    df_encoded[numerical_cols]
)

print("Scaling verification (should be ~0 mean, ~1 std):")
print(f"\nMeans:\n{df_encoded[numerical_cols].mean().round(4)}")
print(f"\nStd Devs:\n{df_encoded[numerical_cols].std().round(4)}")
print(f"\nNote: 'hr' column kept unscaled (range 0-23)")

Scaling verification (should be ~0 mean, ~1 std):

Means:
temp          0.0
atemp        -0.0
hum          -0.0
windspeed     0.0
cnt          -0.0
casual        0.0
registered   -0.0
dtype: float64

Std Devs:
temp          1.0
atemp         1.0
hum           1.0
windspeed     1.0
cnt           1.0
casual        1.0
registered    1.0
dtype: float64

Note: 'hr' column kept unscaled (range 0-23)


## Final Dataset Check

In [21]:
# Final cleaned and processed dataset
print("Dataset Overview:")
print(f"Shape: {df_encoded.shape}")
print(f"\nFirst 5 rows:")
print(df_encoded.head())
print(f"\nData Info:")
df_encoded.info()
print(f"\nBasic Statistics:")
print(df_encoded.describe().round(3))

Dataset Overview:
Shape: (17379, 27)

First 5 rows:
   yr  mnth  hr  holiday  workingday      temp     atemp       hum  windspeed  \
0   0     1   0        0           0 -1.334648 -1.093281  0.947372  -1.553889   
1   0     1   1        0           0 -1.438516 -1.181732  0.895539  -1.553889   
2   0     1   2        0           0 -1.438516 -1.181732  0.895539  -1.553889   
3   0     1   3        0           0 -1.334648 -1.093281  0.636370  -1.553889   
4   0     1   4        0           0 -1.334648 -1.093281  0.636370  -1.553889   

     casual  ...  weathersit_2  weathersit_3  weathersit_4  weekday_0  \
0 -0.662755  ...         False         False         False      False   
1 -0.561343  ...         False         False         False      False   
2 -0.622190  ...         False         False         False      False   
3 -0.662755  ...         False         False         False      False   
4 -0.723603  ...         False         False         False      False   

   weekday_1  weekday_

## Vector Space Representations

In [22]:
# Create feature vectors - each vector focuses on different aspect of behavior

# Weather Impact Vector:
X_weather = df_encoded[["temp", "atemp", "hum", "windspeed", "cnt"]].values

# User Segmentation Vector
X_users = df_encoded[["hr", "casual", "registered"]].values

print(f"X_weather shape: {X_weather.shape} - Weather impact on bike usage")
print(f"X_users shape: {X_users.shape} - User type segmentation by hour")

X_weather shape: (17379, 5) - Weather impact on bike usage
X_users shape: (17379, 3) - User type segmentation by hour


## Repeat for day.csv

In [23]:

df2 = pd.read_csv("bike_sharing_dataset/day.csv")
# Missing values
print(df2.isna().sum())
print("\nMissing values (%):")
print((df2.isna().sum() / len(df2) * 100).round(2))

# Duplicates
print(f"\nTotal duplicate rows: {df2.duplicated().sum()}")
print(f"Duplicates (keep first): {df2.duplicated(keep='first').sum()}")

# Remove duplicates if needed
df2 = df2.drop_duplicates(keep='first')

# Drop non-informative columns
# instant: just a sequence index, no predictive value
# dteday: date string, hour already captures temporal patterns

df2_clean = df2.drop(columns=["instant", "dteday"])

print(f"Columns after cleaning: {df2_clean.columns.tolist()}")
print(f"Shape: {df2_clean.shape}")

# Validate data integrity for data consistency issues: casual users + registered users should equal total count
validation_error = (df2_clean["casual"] + df2_clean["registered"] - df2_clean["cnt"]).abs().sum()

print(f"Data integrity check (should be 0): {validation_error}")

if validation_error == 0:
    print("Data is consistent: casual + registered = cnt")
else:
    print("Data is inconsistent!")

# Encode categorical variables to numerical format
df2_encoded = pd.get_dummies(
    df2_clean,
    columns=["season", "weathersit", "weekday"]
)

numerical_cols = [
    "temp", "atemp", "hum", "windspeed",
    "cnt", "casual", "registered"
]

df2_encoded[numerical_cols] = scaler.fit_transform(
    df2_encoded[numerical_cols]
)

print("Dataset Overview:")
print(f"Shape: {df_encoded.shape}")
print(f"\nFirst 5 rows:")
print(df_encoded.head())
print(f"\nData Info:")
df_encoded.info()
print(f"\nBasic Statistics:")
print(df_encoded.describe().round(3))

instant       0
dteday        0
season        0
yr            0
mnth          0
holiday       0
weekday       0
workingday    0
weathersit    0
temp          0
atemp         0
hum           0
windspeed     0
casual        0
registered    0
cnt           0
dtype: int64

Missing values (%):
instant       0.0
dteday        0.0
season        0.0
yr            0.0
mnth          0.0
holiday       0.0
weekday       0.0
workingday    0.0
weathersit    0.0
temp          0.0
atemp         0.0
hum           0.0
windspeed     0.0
casual        0.0
registered    0.0
cnt           0.0
dtype: float64

Total duplicate rows: 0
Duplicates (keep first): 0
Columns after cleaning: ['season', 'yr', 'mnth', 'holiday', 'weekday', 'workingday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed', 'casual', 'registered', 'cnt']
Shape: (731, 14)
Data integrity check (should be 0): 0
Data is consistent: casual + registered = cnt
Dataset Overview:
Shape: (17379, 27)

First 5 rows:
   yr  mnth  hr  holiday  workingda